In [2]:
import soundfile as sf
import numpy as np
from numba import njit

In [8]:
import soundfile as sf
import numpy as np
from numba import njit


# ----------------------------------------------------------
# Comb filter (Schroeder): delay + feedback + lowpass
# ----------------------------------------------------------
@njit
def comb_filter(x, delay_samples, feedback, damp):
    buf = np.zeros(delay_samples, dtype=np.float32)
    y = np.zeros_like(x)
    idx = 0
    filter_state = 0.0

    for n in range(len(x)):
        bufout = buf[idx]
        # one-pole lowpass inside the loop
        filter_state = (1 - damp) * filter_state + damp * bufout

        y[n] = bufout
        buf[idx] = x[n] + filter_state * feedback

        idx = (idx + 1) % delay_samples

    return y


# ----------------------------------------------------------
# Allpass filter (diffusion)
# ----------------------------------------------------------
@njit
def allpass_filter(x, delay_samples, feedback):
    buf = np.zeros(delay_samples, dtype=np.float32)
    y = np.zeros_like(x)
    idx = 0

    for n in range(len(x)):
        bufout = buf[idx]
        input_sample = x[n]

        # classic allpass equation
        y[n] = -input_sample + bufout
        buf[idx] = input_sample + bufout * feedback

        idx = (idx + 1) % delay_samples

    return y


# ----------------------------------------------------------
# Full reverb mono → stereo
# ----------------------------------------------------------
def process_reverb(x, sr, wet=0.35):
    # Mono conversion if needed
    if x.ndim > 1:
        x = x.mean(axis=1)

    # Parameters
    comb_delays_ms = [29.7, 37.1, 41.1, 43.7]
    ap_delays_ms   = [5.0, 1.7]

    comb_feedback = 0.78
    comb_damp = 0.2
    ap_feedback = 0.60

    # Convert delays to samples
    comb_delays = [int(sr * d/1000) for d in comb_delays_ms]
    ap_delays   = [int(sr * d/1000) for d in ap_delays_ms]

    # Run comb filters
    s = np.zeros_like(x)
    for d in comb_delays:
        s += comb_filter(x, d, comb_feedback, comb_damp)

    # Diffusion via allpasses
    for d in ap_delays:
        s = allpass_filter(s, d, ap_feedback)

    # --- Stereo decorrelation (better than reversing samples) ---
    # Split via slightly different allpass filters
    left  = allpass_filter(s, 7,  0.4)
    right = allpass_filter(s, 11, 0.4)

    # Normalize reverb tail
    tail_gain = 1.0 / np.max(np.abs(np.concatenate([left, right]))) * 0.9
    left  *= tail_gain
    right *= tail_gain

    # --- Wet/Dry mixing ---
    left  = (1 - wet) * x + wet * left
    right = (1 - wet) * x + wet * right

    # Protect against clipping
    mx = max(np.max(np.abs(left)), np.max(np.abs(right)))
    if mx > 1:
        left  /= mx
        right /= mx

    return np.column_stack((left, right))





In [9]:

if __name__ == "__main__":
    input_file = "/Users/marino/Documents/GitHub/fushimi-in-airy/reverb/thisIsAGoodExample.wav"
    output_file = "/Users/marino/Documents/GitHub/fushimi-in-airy/reverb/output_thisIsAGoodExample.wav"

    print("Loading:", input_file)
    x, sr = sf.read(input_file)

    print("Processing reverb...")
    y = process_reverb(x, sr)

    print("Saving:", output_file)
    sf.write(output_file, y, sr)

    print("Done! Listen to:", output_file)

Loading: /Users/marino/Documents/GitHub/fushimi-in-airy/reverb/thisIsAGoodExample.wav
Processing reverb...
Saving: /Users/marino/Documents/GitHub/fushimi-in-airy/reverb/output_thisIsAGoodExample.wav
Done! Listen to: /Users/marino/Documents/GitHub/fushimi-in-airy/reverb/output_thisIsAGoodExample.wav
